In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model
from keras.layers import merge, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping

Test pobranego modelu w całości 

In [3]:
from pathlib import Path
train_dir = "kaggle_bee_vs_wasp/"
bs = 32 # Batch size
resize_size = 224 # for training, resize all the images to a square of this size
training_subsample = 1 # for development, use a small fraction of the entire dataset rater than full dataset

bees_vs_wasps_dataset_path=Path(train_dir) # this is relative to the "example_notebook" folder. Modify this to reflect your setup
df_labels = pd.read_csv(bees_vs_wasps_dataset_path/'labels.csv')
df_labels=df_labels.set_index('id')
# perform dataset subsampling
df_labels = df_labels.sample(frac=training_subsample, axis=0)
insect_class = {'bee': 0,'wasp': 1, 'insect': 2,'other': 3} 
#df_labels.label = [insect_class[item] for item in df_labels.label] 
df_labels = df_labels[['path','label']]

In [4]:
#Uncomment to read data from csv
#df_labels = pd.read_csv("Dataset_full.csv") #full dataset
#df_labels = pd.read_csv("Dataset_10precent.csv") #0.1 fraction for development

In [5]:
train_df, test_val_df = train_test_split(df_labels, test_size=0.2)
test_df, validation_df = train_test_split(test_val_df, test_size=0.5)
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)

In [6]:
#Create custom vgg models

image_input = Input(shape=(224, 224, 3))

# Model with last convolutional layer to train

model1 = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')
last_layer = model1.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
out = Dense(4, activation='softmax', name='output')(x)

model_last_layer = Model(image_input, out, name="model_last_layer")

for layer in model_last_layer.layers[:-6]:
	layer.trainable = False

# Model with 2 last convolutional layers to train

model2 = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')
last_layer = model2.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
out = Dense(4, activation='softmax', name='output')(x)
model_last_2_layers = Model(image_input, out, name="model_last_2_layers")

for layer in model_last_2_layers.layers[:-7]:
	layer.trainable = False

# Full train model

model3 = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')
last_layer = model3.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
out = Dense(4, activation='softmax', name='output')(x)

model_full = Model(image_input, out, name="model_full")



model_last_layer.compile(loss='sparse_categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
model_last_2_layers.compile(loss='sparse_categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
model_full.compile(loss='sparse_categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

model_last_layer.summary()
model_last_2_layers.summary()
model_full.summary()

Model: "model_last_layer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)  

In [7]:
#Create generators from dataframes
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    train_dir,  
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=bs
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validation_df, 
    train_dir, 
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=bs
)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    train_dir, 
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=1
)

Found 9136 validated image filenames belonging to 4 classes.
Found 1143 validated image filenames belonging to 4 classes.
Found 1142 validated image filenames belonging to 4 classes.


In [8]:
#Early stopping
checkpoint1 = ModelCheckpoint(
    "model_last_layer_checkpoint.hdf5", 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=False,
    save_weights_only=False, 
    mode='auto', 
    save_freq='epoch',
    options=None
)
checkpoint2 = ModelCheckpoint(
    "model_last_2_layers_checkpoint.hdf5", 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=False,
    save_weights_only=False, 
    mode='auto', 
    save_freq='epoch',
    options=None
)
checkpoint3 = ModelCheckpoint(
    "model_full_checkpoint.hdf5", 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=False,
    save_weights_only=False, 
    mode='auto', 
    save_freq='epoch',
    options=None
)
early = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=40,
    verbose=1,
    mode="auto",
)

In [48]:
n_training_samples = len(train_df)
n_validation_samples = len(validation_df)

history1 = model_last_layer.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=n_validation_samples//bs,
    steps_per_epoch=n_training_samples//bs,
    callbacks=[checkpoint1, early])

history2 = model_last_2_layers.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=n_validation_samples//bs,
    steps_per_epoch=n_training_samples//bs,
    callbacks=[checkpoint2, early])

history3 = model_full.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=n_validation_samples//bs,
    steps_per_epoch=n_training_samples//bs,
    callbacks=[checkpoint3, early])

Epoch 1/15
285/285 [==============================] - ETA: 0s - loss: 1.1013 - accuracy: 0.5422
Epoch 00001: saving model to model_last_layer_checkpoint.hdf5
285/285 [==============================] - 1430s 5s/step - loss: 1.1013 - accuracy: 0.5422 - val_loss: 0.9532 - val_accuracy: 0.6366
Epoch 2/15
285/285 [==============================] - ETA: 0s - loss: 0.8879 - accuracy: 0.6765
Epoch 00002: saving model to model_last_layer_checkpoint.hdf5
285/285 [==============================] - 1378s 5s/step - loss: 0.8879 - accuracy: 0.6765 - val_loss: 0.7709 - val_accuracy: 0.7241
Epoch 3/15
285/285 [==============================] - ETA: 0s - loss: 0.7586 - accuracy: 0.7256
Epoch 00003: saving model to model_last_layer_checkpoint.hdf5
285/285 [==============================] - 1384s 5s/step - loss: 0.7586 - accuracy: 0.7256 - val_loss: 0.6745 - val_accuracy: 0.7545
Epoch 4/15
285/285 [==============================] - ETA: 0s - loss: 0.6765 - accuracy: 0.7508
Epoch 00004: saving model to mo

In [11]:
model_last_layer = load_model('model_last_layer_checkpoint.hdf5')
filenames = test_generator.filenames
n_test_samples = len(filenames)
scores1 = model_last_layer.evaluate(test_generator,steps=n_test_samples)
print (scores1)

1142/1142 [==============================] - 121s 106ms/step - loss: 0.3630 - accuracy: 0.2767
[0.36303287744522095, 0.2767075300216675]


In [ ]:
#model_last_layer = load_model('model_last_layer_checkpoint.hdf5')

In [22]:
filenames = test_generator.filenames
n_test_samples = len(filenames)
#predict = custom_vgg_model.predict(test_generator, steps=n_test_samples)
scores1 = model_last_layer.evaluate(test_generator,steps=n_test_samples)
scores2 = model_last_2_layers.evaluate(test_generator,steps=n_test_samples)
scores3 = model_full.evaluate(test_generator,steps=n_test_samples)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [1]:
model_last_layer.save('last_layer_trained_model')
np.savetxt("model3_last_layer/val_loss.csv",  
           history1.history["val_loss"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model3_last_layer/val_accuracy.csv",  
           history1.history["val_accuracy"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model3_last_layer/loss.csv",  
           history1.history["loss"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model3_last_layer/accuracy.csv",  
           history1.history["accuracy"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model3_last_layer/accuracy_on_test.csv",  
           scores1, 
           delimiter =", ",  
           fmt ='% s') 

model_last_2_layers.save('last_2_layers_trained_model')
np.savetxt("model4_last_2layers/val_loss.csv",  
           history2.history["val_loss"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model4_last_2layers/val_accuracy.csv",  
           history2.history["val_accuracy"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model4_last_2layers/loss.csv",  
           history2.history["loss"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model4_last_2layers/accuracy.csv",  
           history2.history["accuracy"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model4_last_2layers/accuracy_on_test.csv",  
           scores2, 
           delimiter =", ",  
           fmt ='% s') 

model_full.save('full_trained_model')
np.savetxt("model5_full/val_loss.csv",  
           history3.history["val_loss"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model5_full/val_accuracy.csv",  
           history3.history["val_accuracy"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model5_full/loss.csv",  
           history3.history["loss"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model5_full/accuracy.csv",  
           history3.history["accuracy"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model5_full/accuracy_on_test.csv",  
           scores3, 
           delimiter =", ",  
           fmt ='% s') 

NameError: name 'model_last_layer' is not defined

In [23]:
print ("Model 1 accuracy on test set:" + str(scores1[1]) )
print ("Model 2 accuracy on test set:" + str(scores2[1]) )
print ("Model 3 accuracy on test set:" + str(scores3[1]) )

Model 1 accuracy on test set:0.27272728085517883
Model 2 accuracy on test set:0.1818181872367859
Model 3 accuracy on test set:None
